In [2]:
import pydicom
from pydicom import dcmread
import numpy as np
#import root_numpy
import os
import skimage
import scipy
import pytomography

PyTomography found Apple Silicon GPUs, this is experimental


In [ ]:
class SPOC: 
    def __init__(self, id, num) -> None:
        self.projections = 4
        self.id = 1
        self.num = 0
        self.path = '/.../'
        self.pixelsize = 0.4
        self.type = {
            'experimental' : 0,
            'simulated' : 1
        }

    def _getpath_(self) -> None:
        self.path = os.path.expandvars('../../Dataset')
        images_dir = 'images'
        dicom_dir = 'dicoms'
        exp_path = os.path.join(self.path, 'experimental')
        simul_path = os.path.join(self.path, 'simulated')
        exp_images_path = os.path.join(exp_path, images_dir)
        sim_images_path = os.path.join(simul_path, images_dir)

        dicom_path = os.path.join(self.path, 'dicom')
        dic_images_path = os.path.join(dicom_path, dicom_dir)

        return (exp_images_path, sim_images_path, dic_images_path)
    
    def _getparam_(self, typology: list[str]) -> None:
        print('get parameter from file')
        self.distance = self.focal * 2
        self.angle = 7.
        # take parameter from dicom file
        ds = dcmread(SPOC._getpath_()[self.type[typology]])
        array = ds.pixel_array

        if self.projections == 4:
            return 0


    def import_file(self, typology: list[str] ) -> None:

        if typology == self.type[0]:

             images = [f for f in os.listdir(SPOC._getpath_()[self.type[typology]]) if f.endswith('.txt')]

        elif typology == self.type[1]:

            images = [f for f in os.listdir(SPOC._getpath_()[self.type[typology]]) if f.endswith('.txt') and f.startswith('_')]

        else:
            print('experimental or simulated have to insert')

        return images
    
if __name__ == "__main__":
    file = os.path("\Dataset")
    angles = [0, 45, 90, 135]
    size = 128

    SPOC._getpath_(file)
    ds = dcmread("/Users/nicolaferrara/Downloads/projection_misure_.dcm")
    array_img = ds.pixel_array
    elem_row1 = ds[0x0028, 0x0010]
    elem_row1.value = size
    elem_row2 = ds[0x0028, 0x0011]
    elem_row2.value = size
    vec_data = ds.pixel_array
    vec_data[0] = diff3_0deg
    vec_data[1] = diff3_60deg
    vec_data[2] = diff3_120deg
    vec_data[3] = diff3_180deg


    ds.PixelData = vec_data.tobytes()

In [ ]:
ds = dcmread("/Users/nicolaferrara/Downloads/projection_misure_.dcm")

In [ ]:
from pydicom.util import codify

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy.ndimage import map_coordinates

# Funzione per generare la proiezione 2D di un oggetto 3D
def pinhole_projection_3d(phantom_3d, angle, pinhole_distance, detector_distance, focus_distance):
    """
    Simula una proiezione 2D di un oggetto 3D da un angolo specifico utilizzando un collimatore pinhole.
    
    :param phantom_3d: oggetto 3D (volume) da proiettare
    :param angle: angolo di proiezione
    :param pinhole_distance: distanza tra l'oggetto e il collimatore
    :param detector_distance: distanza tra il collimatore e il rilevatore
    :param focus_distance: distanza dell'oggetto dal fuoco (origine dei raggi)
    :return: proiezione 2D
    """
    # Ruotiamo l'oggetto 3D per simulare la proiezione
    rotated_phantom_3d = rotate(phantom_3d, angle, axes=(0, 1), reshape=True)  # Ruotiamo attorno agli assi X e Y
    
    # Proiezione lungo l'asse z (somma lungo l'asse z)
    projection_2d = np.sum(rotated_phantom_3d, axis=0)
    
    return projection_2d

# Funzione di backprojection 3D (somma lungo la retta)
def pinhole_backprojection_3d(projections, num_angles, phantom_size, pinhole_distance, detector_distance, focus_distance):
    """
    Esegue la backprojection 3D su un insieme di proiezioni per ricostruire un volume 3D.
    
    :param projections: lista di proiezioni 2D
    :param num_angles: numero di angolazioni usate per la ricostruzione
    :param phantom_size: dimensione dell'oggetto (volume) da ricostruire
    :param pinhole_distance: distanza tra l'oggetto e il collimatore
    :param detector_distance: distanza tra il collimatore e il rilevatore
    :param focus_distance: distanza dal fuoco (origine dei raggi)
    :return: volume 3D ricostruito
    """
    reconstructed_volume = np.zeros((phantom_size, phantom_size, phantom_size))
    angles = np.linspace(0, 180, num_angles, endpoint=False)
    
    for i in range(num_angles):
        angle = angles[i]
        projection = projections[i]
        
        # Per ogni voxel (pixel 3D) nel volume ricostruito
        for z in range(phantom_size):
            for x in range(phantom_size):
                for y in range(phantom_size):
                    # Calcolare la distanza tra il voxel e il fuoco
                    dx = x - phantom_size / 2
                    dy = y - phantom_size / 2
                    dz = z - phantom_size / 2
                    
                    # Calcolare la distanza dal rilevatore
                    distance_to_detector = np.sqrt(dx**2 + dy**2 + dz**2) * detector_distance / pinhole_distance
                    
                    # Aggiungere il contributo della proiezione lungo la retta

                    reconstructed_volume[z, y, x] += projection[x] * distance_to_detector

    
    # Normalizzazione del volume ricostruito
    reconstructed_volume /= np.max(reconstructed_volume)
    
    return reconstructed_volume

# Parametri di esempio
phantom_size = 64  # Dimensione del volume 3D
num_angles = 180   # Numero di angolazioni
pinhole_distance = 50  # Distanza tra l'oggetto e il collimatore
detector_distance = 200  # Distanza tra il collimatore e il rilevatore
focus_distance = 100  # Distanza dal fuoco (origine dei raggi)

# Creiamo un oggetto 3D (un cubo)
phantom_3d = np.zeros((phantom_size, phantom_size, phantom_size))
phantom_3d[24:40, 24:40, 24:40] = 1  # Cubo centrale

# Simulazione delle proiezioni 2D usando il collimatore pinhole
projections = []
for angle in range(num_angles):
    projection = pinhole_projection_3d(phantom_3d, angle, pinhole_distance, detector_distance, focus_distance)
    
    # Verifica che ogni proiezione sia della forma corretta (phantom_size, phantom_size)
    projection = np.resize(projection, (phantom_size, phantom_size))  # Assicurati che ogni proiezione sia ridimensionata correttamente
    
    projections.append(projection)

projections = np.array(projections)

# Esegui la backprojection 3D
reconstructed_volume = pinhole_backprojection_3d(projections, num_angles, phantom_size, pinhole_distance, detector_distance, focus_distance)

# Visualizzazione di una sezione trasversale del volume ricostruito
slice_idx = phantom_size // 2
plt.imshow(reconstructed_volume[slice_idx, :, :], cmap='gray')
plt.title(f"Sezione Trasversale del Volume Ricostruito (z={slice_idx})")
plt.colorbar()
plt.show()

ValueError: setting an array element with a sequence.